In [ ]:
import torch as torch
import pandas as pd
import pickle as pickle
import random
import faiss
from nltk.tokenize import sent_tokenize
import keras
import pickle

In [ ]:
user_asp=pd.read_pickle('用户训练集aspect合集')
temp=set()
for i in range(len(user_asp)):
    for j in range(len(user_asp[i])):
        tempppp.add(user_asp[i][j])
len(temp)

In [ ]:
test=pd.read_json('./data/train-data/test_df.json')

In [ ]:
for index,value in test.iterrows():
#     temp=json.loads(value['aspect'])
    user={}
    temp=eval(value['aspect'])
    temp=temp['aspects']
    print(temp)

In [ ]:
uuu=[]       
for i in range(len(temp)):
    user_aspect_emb=user_aspect[i][:2]
    user_aspect_emb=[' '.join(user_aspect_emb)]
    uuu.append(user_aspect_emb)
uuu

In [ ]:
item_ff={}
for i in range(len(user_aspect[0])):
    if 0 in item_ff: #如果存在就直接append
        item_ff[0].extend(user_aspect[0][i])
    else: #不存在，重新创建一个list
        item_ff[0]=user_aspect[0][i]
print(item_ff[0])

In [ ]:
import collections

In [ ]:
user_ff={}
user_aspect_ans={}
count=0
for i in range(len(user_aspect)):
    if len(user_aspect[i])==0:
        user_ff[i]=user_aspect[i]
        continue
    for j in range(len(user_aspect[i])):
        if i in user_ff: #如果存在就直接append
            user_ff[i].extend(user_aspect[i][j])
        else: #不存在，重新创建一个list
            user_ff[i]=user_aspect[i][j]      
    counter = collections.Counter(user_ff[i])
    features_sorted = sorted(counter.items(),key=lambda x:x[1],reverse=True)
    features = []
    for feature in features_sorted:
        features.append(feature[0])
    user_aspect_ans[i]=features
    count=count+1
    if count%1000==0:
        print(count)

In [ ]:
with open('./data/train-data/item_aspect.pkl', "wb") as fOut:
        pickle.dump(user_aspect_ans, fOut)

In [ ]:
item_aspectaaa=pd.read_pickle('./data/train-data/item_aspect.pkl')

In [ ]:

torch.load('./model.pt')

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2').to("cuda")

In [ ]:
tot=pd.read_pickle('./data/TripAdvisor/reviews.pickle')
tot=pd.DataFrame(tot)
tot

In [ ]:
aspect=[]
for i,index in tot.iterrows():
        aspect.append(index['template'][0])

In [ ]:
len(aspect)

In [ ]:
#开始数据处理
totals=pd.read_json('./data/TripAdvisor/Trip_df.json')
totals

In [ ]:
totals

In [ ]:
user_aspect=pd.read_pickle('./data/TripAdvisor/user_aspect.pkl')
item_aspect=pd.read_pickle('./data/TripAdvisor/item_aspect.pkl')

In [ ]:
user_aspect

In [ ]:
ss=[]
for i in range(len(user_aspect)):
    if(len(user_aspect[i])==3):
        print(user_aspect[i])

In [ ]:
with open('user_aspect_top2.pkl', 'wb') as f:
    pickle.dump(plain_user_reviews_dict, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
torch.save(ss, 'user_aspect_top2.pt')

In [ ]:
y = torch.load('user_aspect_top2.pt')

In [ ]:
#数据处理
total=pd.read_json('./data/TripAdvisor/train_df.json')
total.reset_index(inplace=True, drop=True)
total

In [ ]:
for i,index in total.iterrows():
    if i%10000==0:
        print(i)
    if type(index['review'])==float:
        print(index),print(i)

In [ ]:
review=[]
for i,value in total.iterrows():
    review.append(value['template'][2])

In [ ]:
total['template']=review

In [ ]:
total = total[['user', 'item', 'template', 'rating']]
total.columns = ['userId', 'itemId', 'review', 'rating']
total

In [ ]:
total["userId"] = pd.factorize(total["userId"])[0].astype(int)
total["itemId"] = pd.factorize(total["itemId"])[0].astype(int)
total.reset_index(inplace=True, drop=True)
total

In [ ]:
total.to_json('./data/TripAdvisor/Trip_df.json')

In [ ]:
#创建user_toitem列表
user_toitem={}
for i,value in total.iterrows():
    if i%50000==0:
        print(i)
    if value['userId'] not in user_toitem.keys():
        user_toitem[value['userId']]=[]
        user_toitem[value['userId']].append(value['itemId'])
    else:
        user_toitem[value['userId']].append(value['itemId'])
item_touser={}
for i,value in total.iterrows():
    if i%50000==0:
        print(i)
    if value['itemId'] not in item_touser.keys():
        item_touser[value['itemId']]=[]
        item_touser[value['itemId']].append(value['userId'])
    else:
        item_touser[value['itemId']].append(value['userId'])
torch.save(user_toitem,'./data/TripAdvisor/user_toitem.pt')
torch.save(item_touser,'./data/TripAdvisor/item_touser.pt')

In [ ]:
#对user所有的评论，分句后建模索引
user_index_contrust={}
user_index_aspect={}
for i in range(len(user_aspect)):
    if i%500==0:
        print(i)
    temp_u=[]
    for j in range(len(user_aspect[i])):
#         sents_u = sent_tokenize(user_revs[i][j])
        temp_u.append(user_aspect[i][j])
#         temp_u.append(user_revs[i][j])
#     print(temp_u)
    tttt=model.encode(user_aspect[i])
    user_index_aspect[i]=temp_u
    user_index_contrust[i]=tttt
with open('./data/TripAdvisor/user_aspect.pkl', "wb") as fOut:
        pickle.dump({'sentences': user_index_aspect, 'embeddings': user_index_contrust}, fOut)
#对item所有的评论，分句后建模索引
# item_index_contrust={}
# item_index_sent={}
# for i in range(len(item_revs)):
#     if i%1000==0:
#         print(i)
#     temp_u=[]
#     for j in range(len(item_revs[i])):
#         sents_u = sent_tokenize(item_revs[i][j])
#         temp_u.extend(sents_u)
# #         temp_u.append(item_revs[i][j])
#     tttt=model.encode(temp_u)
#     item_index_sent[i]=temp_u
#     item_index_contrust[i]=tttt
# with open('./data/yelp2019/item_index_corpus_emb.pkl', "wb") as fOut:
#         pickle.dump({'sentences': item_index_sent, 'embeddings': item_index_contrust}, fOut)

In [ ]:
for i in range(1):
    if i%500==0:
        print(i)
    temp_u=[]
    for j in range(len(user_aspect[i])):
        sents_u = user_aspect[i]
#         print(user_aspect[0][0])
        temp_u.append(user_aspect[i][j])
    tttt=model.encode(user_aspect[i])
tttt.shape

In [ ]:
temp_u

In [ ]:
tttt[4]

In [ ]:
#对用户所有的评论取平均值，去全局搜索。商品也一样做法
user_rev_emb=[]
for i in range(len(user_revs)):
    if i%500==0:
        print(i)
    li_u=[]
    for j in range(len(user_revs[i])):
        li_u.append(user_revs[i][j])
    tttt=model.encode(li_u)
    tttt=torch.tensor(tttt)
    tttt=tttt.sum(dim=0)/len(user_revs[i])
    user_rev_emb.append(tttt)
user_rev_emb = torch.tensor([item.detach().numpy() for item in user_rev_emb])
user_rev_emb=user_rev_emb.numpy()
user_rev_emb.shape
torch.save(user_rev_emb,'./data/TripAdvisor/user_avg_rev_emb.pt')

#对item所有的aspect，取平均值，作为aspect查询向量
item_rev_aspect_emb=[]
for i in range(len(item_aspect)):
    if i%500==0:
        print(i)
    li_u=[]
    for j in range(len(item_aspect[i])):
        li_u.append(item_aspect[i][j])
    tttt=model.encode(li_u)
    tttt=torch.tensor(tttt)
    tttt=tttt.sum(dim=0)/len(item_aspect[i])
    item_rev_aspect_emb.append(tttt)
item_rev_aspect_emb = torch.tensor([item.detach().numpy() for item in item_rev_aspect_emb])
item_rev_aspect_emb=item_rev_aspect_emb.numpy()
item_rev_aspect_emb.shape
torch.save(item_rev_aspect_emb,'./data/TripAdvisor/item_avg_aspect_emb.pt')

In [ ]:
#建立global索引
reviews=list(train_df['review'])
len(reviews)
temp_u = []
for i in range(len(reviews)):
    if i%50000==0:
        print(i)
#     sents_u = sent_tokenize(reviews[i])
    temp_u.append(reviews[i])
print(len(temp_u))
re_emb=model.encode(temp_u,show_progress_bar=True)
re_emb.shape
with open('./data/train-data/each_sent_emb.pkl', "wb") as fOut:
        pickle.dump({'sentences': temp_u, 'embeddings': re_emb}, fOut)

In [ ]:
corpus_emb=pd.read_pickle('./data/train-data/each_sent_emb.pkl')
temp_u = corpus_emb['sentences']
re_emb = corpus_emb['embeddings']

In [ ]:
#全局检索
print('begin')
quantizer = faiss.IndexFlatL2(384)  # the other index
index = faiss.IndexIVFFlat(quantizer,384, 1000, faiss.METRIC_L2)
index.train(re_emb)
index.add(re_emb)
assert index.is_trained
print("ok")

In [ ]:
print('开始user')
D_u,I_u = index.search(user_rev_emb,3)
user_retrive=[]
for i in range(len(user_revs)):
    if i%500==0:
        print(i)
    temp=''
    for j in range(3):
        temp=temp_u[I_u[i][j]]+temp
    user_temp =model.encode(temp)
    user_retrive.append(user_temp)
user_retrive=torch.tensor(user_retrive)
user_retrive.shape
torch.save(user_retrive,'./data/train-data/user_glabos_retrive.pt')
D_i,I_i = index.search(item_rev_emb,3)
print('开始item')
item_retrive=[]
for i in range(len(item_revs)):
    if i%500==0:
        print(i)
    temp=''
    for j in range(3):
        temp=temp_u[I_i[i][j]]+temp
    item_temp =model.encode(temp)
    item_retrive.append(item_temp)
item_retrive=torch.tensor(item_retrive)
item_retrive.shape
torch.save(item_retrive,'./data/train-data/item_glabos_retrive.pt')

In [ ]:
user_toitem=torch.load('./data/TripAdvisor/user_to_item_list/user_toitem.pt')
# item_revs=pd.read_pickle('./data/TripAdvisor/user_to_item_list/item_reviews.pkl')

In [ ]:
#aspect检索查询
user_rev_By_itemretrive={}
temp_review=[]
for i in range(len(user_count)):
    if i %500==0:
        print(i)
    temp_By_itemretrive={}
    quantizer = faiss.IndexFlatL2(384)  # the other index
    index = faiss.IndexIVFFlat(quantizer,384,1, faiss.METRIC_L2)
    index.train(user_index_contrust[i])
    index.add(user_index_contrust[i])
    D_u,I_u = index.search(item_rev_aspect_emb[user_toitem[i]],2)
#     print(I_u)
    for j in range(len(user_toitem[i])):
        temp=''
        for t in range(2):
            temp=user_index_sent[i][I_u[j][t]]+temp
            temp_emb=model.encode(temp)
        temp_review.append(temp)
        temp_By_itemretrive[user_toitem[i][j]]=temp
#         temp_By_itemretrive[user_toitem[i][j]]=user_index_contrust[i][I_u[j][0]]
    user_rev_By_itemretrive[i]=temp_By_itemretrive

In [ ]:
with open('./data/TripAdvisor/aspect_retrive_top2.pkl', "wb") as fOut:
        pickle.dump(user_rev_By_itemretrive, fOut)

In [ ]:
len(temp_review)

In [ ]:
print(model.max_seq_length)
model.max_seq_length=512
print(model.max_seq_length)
temp_review_emb=model.encode(temp_review,show_progress_bar=True)

In [ ]:
user_rev_By_itemretrive={}
count=0
for i in range(len(user_revs)):
    temp_By_itemretrive={}
    for j in range(len(user_toitem[i])):
        temp_By_itemretrive[user_toitem[i][j]]=temp_review_emb[count]
        count=count+1
    user_rev_By_itemretrive[i]=temp_By_itemretrive
print(count)
torch.save(user_rev_By_itemretrive,'./data/TripAdvisor/user_revemb_By_item_retrive.pt')

In [ ]:
item_rev_By_userretrive={}
count=0
for i in range(len(item_revs)):
    temp_By_itemretrive={}
    for j in range(len(item_touser[i])):
        temp_By_itemretrive[item_touser[i][j]]=temp_review_emb[count]
        count=count+1
    item_rev_By_userretrive[i]=temp_By_itemretrive
print(count)
torch.save(item_rev_By_userretrive,'./data/TripAdvisor/item_revemb_By_user_retrive.pt')